In [ ]:
def schoolbook_mul64(a, b):
    a &= (1 << 64) - 1
    b &= (1 << 64) - 1
    low = 0
    high = 0
    for i in range(64):
        if (a >> i) & 1:
            if i == 0:
                low ^= b
            else:
                low ^= b << i
                high ^= b >> (64 - i)
    mask64 = (1 << 64) - 1
    return [low & mask64, high & mask64]

In [4]:
def limbs_from_int(x, n_limbs):
    """Convert integer x to n_limbs 64-bit limbs (LSB first)."""
    limbs = []
    for _ in range(n_limbs):
        limbs.append(x & ((1 << 64) - 1))
        x >>= 64
    return limbs

def int_from_limbs(limbs):
    """Convert limb list (LSB first) back to Python int."""
    result = 0
    for i, limb in enumerate(limbs):
        result |= limb << (64 * i)
    return result

def schoolbook_mul_nlimbs(a_limbs, b_limbs):
    """
    Multiply two GF(2) polynomials given as lists of 64-bit limbs (LSB first).
    Returns result as list of (2*len - 1) or (2*len) limbs.
    """
    n = len(a_limbs)
    m = len(b_limbs)
    # Result has up to n + m limbs (each 64-bit)
    result = [0] * (n + m)

    for i in range(n):
        for j in range(m):
            # Multiply limbs a[i] * b[j] → 128-bit product
            p_lo, p_hi = schoolbook_mul64(a_limbs[i], b_limbs[j])
            # Add (XOR) into result at position i+j (low) and i+j+1 (high)
            result[i + j] ^= p_lo
            result[i + j + 1] ^= p_hi

    # Mask all limbs to 64 bits (safety)
    mask64 = (1 << 64) - 1
    for k in range(len(result)):
        result[k] &= mask64

    return result

# === Usage for 17669-bit numbers ===
BIT_WIDTH = 17669
N_LIMBS = (BIT_WIDTH + 63) // 64  # = 277

# Example inputs (as Python ints)
a_int = (1 << 17668) | 1          # x^17668 + 1
b_int = (1 << 1000) | 0x12345     # arbitrary

# Convert to 277 limbs (LSB first)
a_limbs = limbs_from_int(a_int, N_LIMBS)
b_limbs = limbs_from_int(b_int, N_LIMBS)

# Multiply
c_limbs = schoolbook_mul_nlimbs(a_limbs, b_limbs)  # length = 554

# Convert back to integer (optional)
c_int = int_from_limbs(c_limbs)

print(f"Result has {len(c_limbs)} limbs = {len(c_limbs)*64} bits")

Result has 554 limbs = 35456 bits


In [5]:
# Example inputs (as Python ints)
a_int = 123
b_int = 123

# Convert to 277 limbs (LSB first)
a_limbs = limbs_from_int(a_int, N_LIMBS)
b_limbs = limbs_from_int(b_int, N_LIMBS)

# Multiply
c_limbs = schoolbook_mul_nlimbs(a_limbs, b_limbs)  # length = 554

print(c_limbs)

[5445, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 